In [37]:
import copy
import numpy as np

class Individuo:
    def __init__(self, alelos, longitud_gen, cromosoma):
        self._alelos = alelos
        self._longitud_gen = longitud_gen
        self._cromosoma = cromosoma
        self._fitness = 0

In [38]:
class AG:
    def __init__(self, cantidad_individuos, alelos, tamano_gen, generaciones, p, problema):
        self._cantidad_individuos = cantidad_individuos
        self._alelos = alelos
        self._tamano_gen = tamano_gen
        self._generaciones = generaciones
        self._p = p
        self._problema = problema
        self._individuos = np.array([])

    def run(self):
        self.crearIndividuos()
        self._mejor_historico = self._individuos[0]
        generacion = 1
        while generacion <= self._generaciones:
            self.evaluaIndividuos()
            hijos = np.array([])
            while len(hijos) < len(self._individuos):
                padre1 = self.ruleta()
                padre2 = self.ruleta()
                while padre1 == padre2:
                    padre2 = self.ruleta()
                h1, h2 = self.cruza(self._individuos[padre1], self._individuos[padre2])
                hijos = np.append(hijos, [h1])
                hijos = np.append(hijos, [h2])
            self.mutacion(hijos)
            self._individuos = np.copy(hijos)
            self._individuos[np.random.randint(len(self._individuos))] = copy.deepcopy(self._mejor_historico)
            if generacion % 200 == 0:
                print(f"Generación: {generacion}\nMejor Histórico: {self._mejor_historico._cromosoma}, Total = {self._mejor_historico._fitness}")
                # Imprimir los tamaños seleccionados
                print("Tamaños seleccionados: ", [self._problema._tamanos[i] for i in range(len(self._mejor_historico._cromosoma)) if self._mejor_historico._cromosoma[i]==1], '\n')
            generacion += 1

    def crearIndividuos(self):
        for i in range(self._cantidad_individuos):
            cromosoma = np.random.randint(2, size = self._alelos)
            individuo = Individuo(self._alelos, self._tamano_gen, cromosoma)
            self._individuos = np.append(self._individuos, [individuo])

    def evaluaIndividuos(self):
        for i in self._individuos:
            i._fitness = self._problema.f(self._tamano_gen, i._cromosoma)
            if i._fitness > self._mejor_historico._fitness:
                self._mejor_historico = copy.deepcopy(i)

    def ruleta(self):
        f_sum = np.sum([i._fitness for i in self._individuos])
        if f_sum < 0:
            f_sum *= -1
        if f_sum == 0:
            return np.random.randint(len(self._individuos))
        else:
            r = np.random.randint(f_sum + 1)
            k = 0
            F = self._individuos[k]._fitness
            if F < 0:
                F *= -1
            while F < r and k < len(self._individuos):
                k += 1
                if self._individuos[k]._fitness < 0:
                    F += self._individuos[k]._fitness * -1
                else:
                    F += self._individuos[k]._fitness
            return k

    def cruza(self, i1, i2):
        h1 = copy.deepcopy(i1)
        h2 = copy.deepcopy(i2)

        s = self._alelos - 1
        punto_cruza = np.random.randint(s) + 1
        h1._cromosoma[punto_cruza:], h2._cromosoma[punto_cruza:] = h2._cromosoma[punto_cruza:], h1._cromosoma[punto_cruza:]
        return h1, h2

    def mutacion(self, hijos):
        for h in hijos:
            for bit in range(len(h._cromosoma)):
                if np.random.rand() < self._p:
                    h._cromosoma[bit] = int(not h._cromosoma[bit])

In [39]:
class pesosSuma:
    def __init__(self, tamanos, limite):
        self._tamanos = tamanos # Arreglo con los tamaños
        self._limite = limite

    def f(self, tamano_gen, cromosoma):
        f = 0
        suma_tamanos = 0
        for i in range(len(cromosoma)):
            if cromosoma[i]:
                f = f + self._tamanos[i] # Sumar los tamaños en las posiciones tomadas
                suma_tamanos = suma_tamanos + self._tamanos[i] # Aumentar peso
        if suma_tamanos <= self._limite: # Si llega al límite, retorna el acumulado
            return f
        else:
            return 0 # Si pasa el límite, retorna 0 para penalizar

In [40]:
def main():
    tamanos = [13, 8, 25, 4, 18, 6, 33, 22, 45, 11, 76, 10, 1]
    limite = 211
    problema = pesosSuma(tamanos, limite)
    alelos = len(tamanos)
    individuos = 32
    tamano_gen = 1 #cada bit representa a un objeto
    generaciones = 2000
    factor_mutacion = 0.01
    ag = AG(individuos, alelos, tamano_gen, generaciones, factor_mutacion, problema)
    ag.run()


In [ ]:
main()